# IQVIA NLP - Content Store Search and Feature Extraction

## API Description
In today’s complex and competitive environment finding the right data to generate the right insight matters. External sources contain vast amounts of rich data but going source by source, searching for and ingesting the right data is time consuming and takes vital time away from developing impactful insights. The IQVIA NLP Content Store APIs are your go-to resource for keeping your team’s focus out of the data weeds and where it belongs: making strategic decisions to achieve your goals.

## Accessing the API
In order to consume this API, you will first need to request access to the Content Store API via this link:
https://api-marketplace.work.iqvia.com/s/communityapi/a085w00000yu12lAAA/external-api-marketplaceiqvianlpcontentstoresearch .

Please refer to "API Documentation" to learn more about accessing and using the API.

## Notebook Description
This notebook is designed to show users an example of using the IQVIA NLP - Content Store Search API.

### Authorization
The instructions for getting your credentials and the API endpoint URL can be found under the section "Get Started" and "How to use the API" following this link: https://api-marketplace.work.iqvia.com/s/communityapi/a085w00000yu12lAAA/external-api-marketplaceiqvianlpcontentstoresearch

In [ ]:
import getpass
import requests

# In this demo scenario, URL for US based customers
# api_marketplace_url = 'https://vt.us-rds.solutions.iqvia.com/easl/api/v1/easl'
# In this demo scenario, URL for EU based customers
api_marketplace_url = 'https://vt.eu-apim.solutions.iqvia.com/eu/easl/api/v1/easl'

mkp_user = input("Marketplace clientId: ")
mkp_password = getpass.getpass("Marketplace clientSecret: ")
mkp_headers = {'clientId': mkp_user, 'clientSecret': mkp_password}

# Check credentials by making a dummy request
print("Thanks for inputting your user name and password!")

### Example: Make a request with I2E query on selected I2E index
Content Store Search API expects an I2E query and an Index name as input parameter. This example shows how to make a request to the API with a pre-configured API from  as input.

In [ ]:
import requests
import os

# Define input query and index to run
input_query = os.path.join(os.getcwd(), "demo_queries/Content_Store_Search-Post_EASL_Run/biomkr_reln-covid19_disease.i2qy")
index_name = "medline-full"

# Make a request
print("Posting query request...")
with open(input_query, "r") as query:
    response = requests.post(api_marketplace_url, headers=mkp_headers, files=dict(query=(input_query, query, "application/vnd.linguamatics.i2e.i2qy")), data=dict(index_name=index_name))

In [ ]:
# Check the response
if response.status_code == 200:
    print("Success!")
    body_json = response.json()
    print(f"Raw JSON response from the API is: {body_json}")
else:
    raise Exception(f"Error: {response}")

Now that we have got the JSON response from the Post EASL Run API, we could get the response with the GET endpoint for the run id returned.

In [ ]:
run_id = body_json["id"]
run_url = api_marketplace_url+"/"+run_id

response = requests.get(run_url, headers=mkp_headers)

# Check the response
if response.status_code == 200:
    print("Success!")
    body = response.json()
    print(f"Raw JSON response from the API is: {body}")
else:
    raise Exception(f"Error: {response}")


Get the list of document ID returned by the API.

In [ ]:
doc_id_list = [result["Doc"] for result in body["results"]]
print(f"Document IDs are: {doc_id_list}")

Fetch features from the first two documents in the list.

In [ ]:
features = ["chebi", "meddra"]
results_json = []
fetch_url = f"https://vt.eu-apim.solutions.iqvia.com/eu/fetch-features/api/v1/fetch-features"
print(f"Making requests to get features for specified documents...")
for feature in features:
    response = requests.get(fetch_url, headers=mkp_headers, params={"index": index_name, "features": feature, "docIds": doc_id_list[:2]})
    # Check the response
    if response.status_code == 200:
        print("Success!")
        json_response = response.json()
        results_json.append(json_response)
    else:
        raise Exception(f"Error: {response}")
print(f"Raw JSON response from the API for documents of interest: {results_json}")

Convert to pandas dataframe

In [ ]:
import pandas as pd

# initiate an empty dataframe
df = pd.DataFrame()
pd.set_option("display.max_rows", None, "display.max_columns", None, "display.width", 1000)

# Retrieve main results from the JSON response, please note this cell would fail if the request failed in the last step

for result_list in results_json:
    for result_dict in result_list:
        results = result_dict["results"]
        df_dict = {"doc_id": result_dict["doc_id"]}
        for output_dict in results:
            for key, value_dict in output_dict.items():
                df_dict[key] = value_dict['value']
        df = pd.concat([df, pd.DataFrame.from_records([{**df_dict}])], ignore_index=True)

# df.to_csv("content_store_search_output_demo.csv", index=False)

# Check the dataframe
df

That's it! Hope you find this tutorial useful! Bye!